<div style="padding: 20px; text-align: center; color: white;">
    <div>
        <h1 style="margin: 10px 0;"><strong>Introduction to Agentic AI with LangGraph</strong></h1>
        <h2>Matthew Sayer, AI Engineer</h2>
    </div>
</div>


---
## <span style="color: #ffffff;"><strong>Part 1:</strong></span> What is Agentic AI and LangGraph?

### **Agentic AI**: An introduction

<strong>Agentic AI refers to AI systems designed to function as autonomous "agents" that can:</strong>

1. Plan and execute complex tasks by breaking them into logical steps
2. Make decisions independently based on goals, context, and available information provided by the system and user prompts
3. Adapt dynamically to changing conditions and unexpected outcomes, through the use of LLM-driven routing
4. Self-improve through feedback loops and iterative approaches
5. Utilise various tools and capabilities as needed to accomplish objectives
6. Unlike traditional conversational AI flows that simply respond to user prompts with static outputs, agentic AI systems actively manage their own workflows, determine what information they need, decide which actions to take, and persist until objectives are achieved.

### Key Components of Agentic Systems
1. Memory mechanisms that maintain context across multiple steps, stored in a State
2. Reasoning capabilities to evaluate options and make decisions, powered by LLMs
3. Planning functions to break complex tasks into achievable steps, through structured Nodes
4. Tool integration to expand capabilities far beyond language processing

#### Traditional LLM applications follow a simple pattern:

User provides prompt → LLM generates response → Interaction ends

#### Agentic systems transform this into a continuous process:

User defines goal → Agent plans approach → Agent takes actions → Agent evaluates results → Agent adapts and continues → Goal achieved

This shift from single-turn to multi-turn, goal-oriented workflows represents a fundamental advancement in how AI systems operate and the problems they can solve.


### **LangGraph**: The Framework Behind the Workflow
LangGraph is a framework developed by LangChain that allows users to represent Large Language Model (LLM) systems as state machine graphs. This framework provides a simple yet powerful representation that offers a clear workflow for navigating complex tasks.

In LangGraph, each node in the graph updates a shared graph state, which is how information is passed from one node to another. The output of the graph is the resulting state once the end node is reached.

While there are advanced techniques that allow you to stream elements internally from the graph, we will not be covering those in this example notebook.


### Objectives of this Notebook

In this notebook, you will:

1. Learn how to set up a LangGraph.
2. Use an LLM (Large Language Model) to retrieve and summarise information.
3. Build an agentic flow that can perform conversational AI requirements, intent design, utterance generation and conversational flow design.

This step-by-step guide will help you understand the key components of agentic AI and show you how to construct workflows that can handle complex, adaptive tasks.


### The Research Agent Graph Architecture

Below is the graph we will be creating in this notebook. This graph represents a conversation design agent with specialised components:

1. Requirements Scientist - Analyses business requirements and extracts key needs
2. Intent Master - Identifies and categorises core user intents
3. Utterance Wizard - Generates diverse training phrases for each intent
4. Conversational Artist - Crafts engaging, contextually appropriate responses
5. Combiner - Collects all of the team outputs, and combines into a readable format for the user

<div style="text-align: center; padding:20px;"> <img src="./assets/agenticbasicarchitecture.png" alt="Conversation Design Agent Architecture" style="width: 75%; border-radius: 50px;"> </div>

## The Agent Workflow
1. Takes business requirements as input
2. Analyses and extracts core user needs and goals
3. Identifies key intents users would have when interacting with the system
4. Generates diverse ways users might express each intent
5. Creates natural, engaging responses for each intent


### Let’s get started!

---
## <span style="color: #ffffff;"><strong>Part 2:</strong></span> Setup

### Creating and Activating a Virtual Environment

To ensure that all dependencies are installed in an isolated environment, it is recommended to create a virtual environment. Follow the steps below:
 
1. **Create a virtual environment**:

> ```bash
> uv sync
> ```

This command will create a directory named `.venv` in your current working directory, and it will create a uv.lock file which will track the packages you have installed.

2. **Set the notebook kernel to the virtual environment**:

To use the virtual environment as the kernel for your Jupyter notebook, follow these steps:

> ```bash
> uv run python -m ipykernel install --user --name=venv
> ```

After running the above commands, you can select the `venv` kernel in your Jupyter notebook interface.

3. **Set up your Ollama model**:

Download the ollama model of your choice, we can use TinyLlama in this example.

> ```bash
> ollama pull tinyllama:latest
> ```




### Installing Dependencies

The uv sync command has already handled the installation of your dependencies, and you can see them in the pyproject.toml file under the dependencies list. These include the necessary langchain components, ollama and the misc packages for this notebook.

### Set up your Ollama model
This cell sets up the connection to your Ollama model which will be used for the agentic transactions.

In [15]:
from langchain_ollama import ChatOllama

model_name = "tinyllama:latest" #Replace with the model of your choice (as above in the pull)

llm = ChatOllama(model=model_name, temperature=0.5)

print(f"Initialised {model_name}.")

Initialised tinyllama:latest.


---
## <span style="color: #ffffff;"><strong>Part 3:</strong></span> Building the Agent Graph

### Define the shared graph state

Before diving into the details, let's understand what a graph is in the context of LangGraph. A graph is essentially a structured workflow of interconnected components (nodes) that work together to solve a problem. Each node performs a specific function or task, and the connections between nodes (edges) define how information flows through the system.

> **NOTE**: Key points about the graph state object
> 1. It's shared across all nodes in the graph.
> 2. It serves as a centralised data store for the entire system.
> 3. Any information that needs to be passed between nodes must be updated in the state.
> 4. Each node can read from and write to the state, allowing for complex information flow.

In [3]:
from typing import Optional, TypedDict

class ConversationalAgentState(TypedDict):
    input: str  # The initial business requirements from the user
    requirements_analysis: str # Output from requirements_scientist_node
    intent_model: str  # Output from intent_master_node
    utterances: str # Output from utterance_wizard_node
    conversational_flows: str  # Output from conversational_artist_node
    output: str  # Final combined output
    error: Optional[str]  # Error information if any
    loop: int  # Counter to prevent infinite recursion

### Define Utils

These functions will help to ensure that our LLM JSON responses are properly processed at each stage.

In [4]:
from colorama import Fore as colour
import json
from typing import Any, Optional

def stream_llm_response(prompt: str, 
                       llm: Any, 
                       color_code: str = colour.RESET, 
                       prefix: str = "",
                       stop_at: Optional[str] = None) -> str:
    """Stream response from LLM with coloured output."""
    print(color_code + f"{prefix}", end="")
    response = ""
    for chunk in llm.stream(prompt):
        chunk_text = chunk.content if hasattr(chunk, "content") else str(chunk)
        response += chunk_text
        print(colour.RESET + chunk_text, end="")
        if stop_at and stop_at in chunk_text:
            break
    return response

### Creating the <span style="color: #ffffff;"><strong>Requirements Scientist</strong></span> Node

This function defines the **first node** in our graph. This node is where we will be **generating requirements based on the user input** that will be used by later nodes to retrieve information.

> **NOTE**: A `node` is essentially a function that takes the graph state as an input, performs some operations, and updates the state accordingly by returning a dictionary where the `key` matches a variable in the graph state class we defined above. You can think of it analogously as an agent in a team.
> 
> In this case, the `requirements_scientist` node uses the user input from the graph state, and returns a set of generated search queries based on this input.

In [5]:
def requirements_scientist_node(state: dict):
    print(colour.CYAN + "\n[Node: Requirements Scientist] Analysing business requirements...")

    input_requirements = state.get("input", "")
    
    prompt = f"""Analyse these business requirements: {input_requirements}

Please provide a clear analysis covering:
- Primary goal of the system
- Key user needs
- Functional requirements
- Any constraints or limitations

Write your analysis in clear, structured text."""

    response = stream_llm_response(
        prompt=prompt,
        llm=llm,
        color_code=colour.CYAN,
        prefix="Analysing Requirements: "
    )
    
    print(colour.CYAN + "\n✓ Requirements analysis complete!")
    return {"requirements_analysis": response, "loop": 1}

### Creating the <span style="color: #ffffff;"><strong>Intent Master</strong></span> Node

This node is responsible for taking the requirements generated by the requirements scientist, and creating an intent model based on that. The intent model is then stored in the graph state to be accessed by subsequent nodes (agents).

In [6]:
def intent_master_node(state: dict):
    print(colour.BLUE + "\n[Node: Intent Master] Generating intent model...")

    requirements = state.get("requirements_analysis", "")
    
    prompt = f"""Based on these requirements:
{requirements}

Identify the key intents (user goals) for this conversational AI system. For each intent, describe:
- Intent name
- What the user is trying to accomplish
- Why this intent is important

Write your response as a clear list of intents with descriptions."""

    response = stream_llm_response(
        prompt=prompt,
        llm=llm,
        color_code=colour.BLUE,
        prefix="Generating Intents: "
    )
    
    print(colour.BLUE + "\n✓ Intent model generation complete!")
    return {"intent_model": response, "loop": state.get("loop", 0) + 1}

### Creating the <span style="color: #ffffff;"><strong>Utterance Wizard</strong></span> Node

This node is responsible for creating example utterances for each intent in our model.

In [7]:
def utterance_wizard_node(state: dict):
    print(colour.YELLOW + "\n[Node: Utterance Wizard] Generating example utterances...")

    intent_model = state.get("intent_model", "")
    
    if not intent_model:
        error_msg = "Error: No intent model found in state"
        print(colour.RED + f"\n{error_msg}")
        return {"error": error_msg, "loop": state.get("loop", 0) + 1}
    
    prompt = f"""Based on this intent model:
{intent_model}

Generate 5 example utterances for each intent. Show how users might naturally express each intent in different ways.

Make the utterances:
- Natural and conversational
- Varied in length and style
- Representative of real user speech
- Include some casual language

Format as a clear list for each intent."""

    response = stream_llm_response(
        prompt=prompt,
        llm=llm,
        color_code=colour.YELLOW,
        prefix="Generating Utterances: "
    )
    
    print(colour.YELLOW + "\n✓ Utterance generation complete!")
    return {"utterances": response, "loop": state.get("loop", 0) + 1}

### Creating the <span style="color: #ffffff;"><strong>Conversational Artist</strong></span> Node

This node is responsible for creating conversational flow examples based on our requirements, intents and utterances.

In [8]:
def conversational_artist_node(state: dict):
    print(colour.MAGENTA + "\n[Node: Conversational Artist] Creating conversational flows...")

    utterances = state.get("utterances", "")
    
    if not utterances:
        error_msg = "Error: No utterances found in state"
        print(colour.RED + f"\n{error_msg}")
        return {"error": error_msg, "loop": state.get("loop", 0) + 1}
    
    prompt = f"""Based on these utterance examples:
{utterances}

Create sample conversations showing how a bot should respond to each type of user input.

For each intent, show:
- User says something (from the utterances)
- Bot responds helpfully
- User might ask a follow-up
- Bot provides more detail

Make the bot sound helpful, friendly, and natural. Keep responses concise but informative."""

    response = stream_llm_response(
        prompt=prompt,
        llm=llm,
        color_code=colour.MAGENTA,
        prefix="Generating Conversational Flows: "
    )
    
    print(colour.MAGENTA + "\n✓ Conversational flows generation complete!")
    return {"conversational_flows": response, "loop": state.get("loop", 0) + 1}

### Creating the <span style="color: #ffffff;"><strong>Combiner</strong></span> Node

Once we have verified that we have collected enough information to answer the user's question, the **combiner node** combines our requirements, intents, utterances and conversational designs into a readable format.

In [9]:
def combiner_node(state: dict):
    print(colour.MAGENTA + "\n[Node: Combiner] Creating final conversational design document...")

    # Extract all the text inputs
    input_requirements = state.get("input", "")
    requirements_analysis = state.get("requirements_analysis", "")
    intent_model = state.get("intent_model", "")
    utterances = state.get("utterances", "")
    conversational_flows = state.get("conversational_flows", "")
    error = state.get("error", "")

    if error:
        print(colour.RED + f"\nPrevious error detected: {error}")
        print(colour.RED + "Proceeding with available data...\n")

    prompt = f"""Create a well-structured conversation design document combining all these elements:

ORIGINAL REQUIREMENTS:
{input_requirements}

REQUIREMENTS ANALYSIS:
{requirements_analysis}

INTENT MODEL:
{intent_model}

UTTERANCE EXAMPLES:
{utterances}

CONVERSATION FLOWS:
{conversational_flows}

Format this as a professional document with clear sections:
1. Executive Summary
2. Conversation Design Overview  
3. Intent Structure
4. Sample Conversations
5. Implementation Recommendations

Make it comprehensive and ready to present to stakeholders."""

    response = stream_llm_response(
        prompt=prompt,
        llm=llm,
        color_code=colour.MAGENTA,
        prefix="Creating Final Document: "
    )
    
    print(colour.MAGENTA + "\n✓ Conversation design document complete!")
    return {"output": response, "loop": state.get("loop", 0) + 1}

### Designing the Agent Graph

Finally, after defining each of our graphs' nodes, we can construct the graph by adding each node to the graph object.

Here we are arranging our nodes and defining their relationship to one another.

Below is a reminder of our architecture:

<div style="text-align: center; padding:20px;">
    <img src="./assets/agenticbasicarchitecture.png" alt="Conversational Design Agent Graph Architecture" style="width: 75%; border-radius: 50px;">
</div>

> **NOTE**: 
> - The solid arrows between the nodes on the graphs are `edges`. These represent one-way connections between two nodes, indicating the flow from one node to another.


In [10]:
from langgraph.graph import StateGraph, START, END

# Instantiate the graph object with our updated state class
graph = StateGraph(ConversationalAgentState)

# Add the nodes we have created previously, providing a name string for each
graph.add_node(node="requirements_scientist", action=requirements_scientist_node)
graph.add_node(node="intent_master", action=intent_master_node)
graph.add_node(node="utterance_wizard", action=utterance_wizard_node)
graph.add_node(node="conversational_artist", action=conversational_artist_node)
graph.add_node(node="combiner", action=combiner_node)

# The edges remain the same
graph.add_edge(START, "requirements_scientist")
graph.add_edge("requirements_scientist", "intent_master")
graph.add_edge("intent_master", "utterance_wizard")
graph.add_edge("utterance_wizard", "conversational_artist")
graph.add_edge("conversational_artist", "combiner")
graph.add_edge("combiner", END)

# Compile the graph
graph = graph.compile()

Congratulations! You have successfully built the research agent graph. In the next section, we will run the graph and see how it performs.

---
## <span style="color: #ffffff;"><strong>Part 4:</strong></span> Running the Agent Graph

### Input a query

This is the query you want the Conversational Agent to create requirements, intents, utterances and conversational designs for. Feel free to change the query to test the agent's capabilities.

In [11]:
user_query = "Generate a basic flow for a chatbot that can answer questions about the weather."

### Build and Run the Graph

The **running the graph** cell is where the research agent’s entire **workflow** is executed, demonstrating how the agent navigates through the graph to answer the user's query.

The graph returns the final state of the system, which includes the summarised information that the agent has collected.

In [12]:
final_state = graph.invoke({"input": user_query})

print(colour.WHITE + "\n[Final Graph State Output]:\n\n", json.dumps(final_state, indent=2))


[Node: Requirements Scientist] Analysing business requirements...
Analysing Requirements: Analysis for Chatbot to Answer Weather Questions

Primary Goal: To provide users with accurate and up-to-date weather information
Key User Needs: To receive reliable weather forecasts and updates
Functional Requirements:
1. Ability to ask about the current weather conditions, including temperature, precipitation, and wind speed
2. Accessibility of the chatbot for users who may not have access to a traditional weather app or website
3. The ability to provide real-time updates on any changes in weather patterns or conditions
4. The option to receive notifications about severe weather events (such as thunderstorms, tornadoes, and flash floods)
5. The ability to connect the chatbot with local weather information sources (such as weather stations, news websites, or weather apps) for more comprehensive and accurate weather forecasts

Constraints or Limitations:
1. Limited access to real-time informatio

#### Congratulations! You have successfully built a conversational development agentic flow using LangGraph!

---
## <span style="color:#ffffff;"><strong>Part 5:</strong></span> Summary

### Summary of Accomplishments:
- Developed a Conversational Project agent using LangGraph in an Agentic AI architecture.
- Enabled the agents to generate requirements, intents, utterances and conversational designs.
- Demonstrated the use of nodes and graphs to create agentic stages, with edges directing a sequential execution.

> Author: Matthew Sayer